In [2]:
import os
import sys
from pydub import AudioSegment
from google.colab import files

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [3]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q transformers torch pydub



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
class AudioAnalyzer:

    def __init__(self, whisper_model_name="tiny"):

        try:
            print(f"Loading Whispermodel: {whisper_model_name}...")
            self.whisper_model = whisper.load_model(whisper_model_name)
            print("Whisper model loaded ")

            print("Initializing sentiment analysis")
            self.sentiment_pipeline = pipeline("sentiment-analysis")
            print("Sentiment analysis pipeline is installed .")
        except Exception as e:
            print(f"Error initializing models: {e}")
            raise

    def  process_audio(self, audio_path):

        try:

            print(" Transcribing and identifying language.")
            whisper_result = self.whisper_model.transcribe(audio_path)
            transcription = whisper_result["text"]
            language = whisper_result["language"]


            print(" Performing sentiment analysis")
            sentiment_result = self.sentiment_pipeline(transcription)

            return {
                "language": language,
                "transcription": transcription,
                "sentiment_label": sentiment_result[0]['label'],
                "sentiment_score": sentiment_result[0]['score']
            }
        except Exception as e:
            print(f"An error occurred during audio processing: {e}")
            return None

In [ ]:
def main():

    print(" AI-Powered Audio Analysis Tool")
    print("Please upload your audio file ")
    uploaded = files.upload()
    if not uploaded:
        print("No file was uploaded.")
        return

    file_name = next(iter(uploaded.keys()))
    print(f"Uploaded '{file_name}'")

    base, ext = os.path.splitext(file_name)
    if ext.lower() != '.wav':
            try:
                print("Converting audio file to WAV format for best results...")
                audio = AudioSegment.from_file(file_name)
                wav_path = f"{base}.wav"
                audio.export(wav_path, format="wav")
                file_name = wav_path
                print("Conversion successful.")
            except Exception as e:
                print(f"Failed to convert audio file: {e}. Please ensure it is not corrupted.")
                return



    analyzer = AudioAnalyzer()
    analysis_results = analyzer.process_audio(file_name)

    if analysis_results:

            print("\n" + "="*50)
            print("   Final Analysis Report")
            print("="*50)
            print(f"[*] Language Detected: {analysis_results['language'].upper()}")
            print("-" * 50)
            print("[*] Full Transcription:")
            print(analysis_results['transcription'])
            print("-" * 50)
            print("[*] Sentiment Analysis:")
            print(f"    - Label: {analysis_results['sentiment_label']}")
            print(f"    - Score: {analysis_results['sentiment_score']:.4f}")
            print("="*50)

if  __name__ == "__main__":
    main()

 AI-Powered Audio Analysis Tool
Please upload your audio file 
